# FigureYa104GEOmining

title: "FigureYa104GEOmining"

author: "Yu Sun, Xuan Da, Taojun Ye"

reviewer: "Ying Ge"

date: "2025-5-20"

output: html_document

## 需求描述

在GEO数据库中检索到的高通量数据，想批量获得它们出自哪篇文章，标注影响因子，输出文本文件和网页，网页文件里有链接到每篇文章的pubmed页面。

**用法参考这篇帖子：**<https://mp.weixin.qq.com/s/G-CQhNEJBmMRuDe2kxND_w>

## Requirement Description

Retrieve high-throughput data from the GEO database and obtain information on which articles they are from in bulk, annotate the impact factors, and output text files and web pages. The web page files should include links to the PubMed pages of each article. **For usage, refer to this post:** <https://mp.weixin.qq.com/s/G-CQhNEJBmMRuDe2kxND_w>

## 应用场景

场景一：老板让测序，怎样设计实验呢？参考测同样数据的文章是怎样设计实验的。

场景二：测序数据回来了，怎样分析？能画哪些图？结果怎样描述？参考类似的数据的文章吧！

场景三：想结合已发表的数据做整合分析，哪套数据更靠谱？先看影响因子高的文章里的数据吧！

## Application Scenario

Scenario 1: The boss asks for sequencing. How to design the experiment? Refer to how the experiment is designed in the article that measures the same data.

Scenario 2: The sequencing data is back. How to analyze it? What kind of graphs can be drawn? How to describe the results? Refer to articles with similar data!

Scenario 3: If you want to conduct a meta-analysis based on published data, which set of data is more reliable? Let's start with the data from articles with high impact factors!

## 环境设置

下载并安装Anaconda发行版，https://www.anaconda.com/distribution/#download-section

里面已经包含了运行本文档所需的Python3、ipython、Jupyter notebook。

需要额外安装BioPython、metapub和pytablewriter，由于eutils在比较新的版本中更新了API导致不能向前兼容，所以也需要重新安装一个比较旧的版本，在终端运行以下命令来安装：

## Environment setup
Download and install the Anaconda distribution from https://www.anaconda.com/distribution/#download-section
It already includes Python 3, ipython, and Jupyter notebook required to run this document.
Additional installations of BioPython, metapub, and pytablewriter are required. Since the API of eutils has been updated in newer versions, making them incompatible with previous versions, it is necessary to reinstall an older version. To install, run the following command in the terminal:

```bash
conda install -c anaconda biopython
pip install metapub
pip install eutils==0.5.0
pip install pytablewriter
pip install markdown
pip install tqdm # Anaconda自带tqdm库# Anaconda comes with the tqdm library
```

## 输入

把要检索的关键词写进下面代码区的”term = “的后面，例如`(gds pubmed[Filter]) AND "Drosophila melanogaster"[orgn:__txid7227] AND ATAC-seq`。

建议先在<https://www.ncbi.nlm.nih.gov/geo/>网站上检索，尝试好合适的关键词后，再来提取文献。

开头加上`(gds pubmed[Filter])` ，就会过滤掉那些还没有发表文章的数据。

##Input
Input the keyword you want to search into the code area below after "term = ", for example, `(gds pubmed[Filter]) AND "Drosophila melanogaster"[orgn:__txid7227] AND ATAC-seq`.
It is recommended to first search on the website <https://www.ncbi.nlm.nih.gov/geo/> and try using appropriate keywords before extracting relevant literature.
Adding `(gds pubmed[Filter])` at the beginning will filter out data for those who have not yet published articles.

## 运行代码

打开Anaconda——Jupyter Notebook，打开FigureYa104GEOmining.ipynb文档，在Jupyter Notebook中点击Run按钮，运行下面的代码。

**加速：**建议自己注册一个NCBI的账号，然后点击右上角自己的邮箱，申请API key，以加快检索速度：E-utils users are allowed 3 requests/second without an API key. Create an API key to increase your e-utils limit to 10 requests/second。

把你的API key添加到代码区的“Entrez.api_key = ”后面

API key的获取方法详见<https://ncbiinsights.ncbi.nlm.nih.gov/2017/11/02/new-api-keys-for-the-e-utilities/>

## Running the code
Open Anaconda - Jupyter Notebook, open the FigureYa104GEOmining.ipynb document, and click the "Run" button in Jupyter Notebook to run the code below.
**Speeding up:** It is recommended to register for an NCBI account and then click on your email address in the top right corner to apply for an API key, in order to expedite the retrieval process: E-utils users are allowed 3 requests/second without an API key. Creating an API key will increase your e-utils limit to 10 requests/second.
Add your API key after "Entrez.api_key = " in the code area
For detailed information on how to obtain an API key, please refer to <https://ncbiinsights.ncbi.nlm.nih.gov/2017/11/02/new-api-keys-for-the-e-utilities/>

## 下面是代码区

## Below is the code area

In [ ]:
from Bio import Entrez
from bs4 import BeautifulSoup
from urllib.request import urlopen
from metapub import PubMedFetcher
import re
from pytablewriter import MarkdownTableWriter
import sys
from IPython.display import display, Markdown
from markdown import markdown
from tqdm import tqdm

# Python的默认迭代深度上限太低啦！基于BS4的爬虫在解析过程中很容易报错的
# The default iteration depth limit of Python is too low! Crawlers based on BS4 are prone to errors during the parsing process
sys.setrecursionlimit(999999)

# 如果你还没申请自己的NCBI账号，就暂时先用作者的email
# If you haven't applied for your own NCBI account, you can temporarily use the author's email
Entrez.email = "suny226@mail2.sysu.edu.cn"
# 如果申请了API key，就把它粘贴到 = 后面
# If you have applied for an API key, paste it after the =
#Entrez.api_key = "yourAPIkey"

# 把你要检索的关键词写在这里
# Write the keyword you want to search here
#term = "(gds pubmed[Filter]) AND Drosophila melanogaster[porgn:__txid7227] AND ATAC-seq"
#term = "(gds pubmed[Filter]) AND Oryza sativa[porgn:__txid4530] AND ChIP-seq"
#term = "(gds pubmed[Filter]) AND Arabidopsis thaliana[porgn:__txid3702] AND ChIP-seq"
term = "(gds pubmed[Filter]) AND Homo sapiens[porgn:__txid9606] AND ChIP-seq AND YAP"

handle = Entrez.esearch(db="gds",term=term, retmax = 100000)
record = Entrez.read(handle)

fetch = PubMedFetcher()
journal_sep_regex = re.compile(r' |\. ')

writer = MarkdownTableWriter()
writer.headers = ["GEO accession", "PMID", "Journal", "2017/2018 Impact Factor", "Year", "Title", "Authors"]
writer.value_matrix = []

print("The key words you searched with return {} results. Please wait parsing...".format(len(record['IdList'])))

with open('GEO_citations.txt', 'w') as f:
    for id in tqdm(record['IdList']):
        url = 'https://www.ncbi.nlm.nih.gov/gds/?term=' + id
        soup = BeautifulSoup(urlopen(url), 'html.parser')
        accn = soup.body.form('dl', class_="rprtid")[0].contents[1].string
        accn_url = 'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=' + accn
        soup = BeautifulSoup(urlopen(accn_url), 'html.parser')
        pmids = soup.body.find('span', class_ = "pubmed_id")['id'].split(',')
        for pmid in pmids:
            pmid_url = 'https://www.ncbi.nlm.nih.gov/pubmed/' + pmid
            article = fetch.article_by_pmid(pmid)

            # 下面这个网址是letpub用来抓取影响因子的，应该是很靠谱了
            # The following URL is used by letpub to retrieve impact factors, so it should be quite reliable
            url = 'https://www.scijournal.org/impact-factor-of-' + journal_sep_regex.sub("-", article.journal).upper().rstrip('.') + ".shtml"
            try:
                soup = BeautifulSoup(urlopen(url), 'html.parser')
                if_str = soup.body.find(text = re.compile('2017/2018 Impact Factor.+'))
                if_value = if_str.replace('2017/2018 Impact Factor : ', "")
                #该网站上有些期刊的2017/2018影响因子处为NA，可以把2017/2018改为2016
                # Some journals on this website have their 2017/2018 impact factors marked as NA. You can replace 2017/2018 with 2016
                #if_str = soup.body.find(text = re.compile('2016 Impact Factor.+'))
                #if_value = if_str.replace('2016 Impact Factor : ', "")
            except:
                if_str = 'NA'
                if_value = '-NA-'
            author_list = ", ".join(article.authors[:-1]) + " and " + article.authors[-1]
            f.write("{}: {}. {} {} {} {}\n".format(accn, author_list, article.title, article.journal, article.year, if_str))
            writer.value_matrix.append(['[' + accn + '](' + accn_url + ')', '[' + pmid + '](' + pmid_url + ')', article.journal, if_value, article.year, article.title, author_list])

with open('GEO_citations.html', 'w') as f:
    table_html = markdown(writer.dumps(), extensions = ['markdown.extensions.tables'])
    content = """<html>
<head></head>
<body>{}</body>
</html>""".format(table_html)
    f.write(content)


## 输出

在当前文件夹里生成分两个文件：

1. `GEO_citations.txt`：文本文件，包含文章信息的汇总

2. `GEO_citations.html`：网页中嵌入的三线表，蓝色字带链接，点击GSE ID可直达数据的GEO页面，点击PMID可直达文章的Pubmed页面。默认为按照相关性排序“Sort by Default order”，可以复制到Excel中自行排序、筛选等操作；链接也会保留到Excel文件中，点击链接可直接跳转至paper网页。

##Output
Generate two files in the current folder:
1. `GEO_citations.txt`: a text file containing a summary of article information
2. `GEO_citations.html`: The embedded three-line table in the webpage, with blue text and links. Clicking on the GSE ID will directly take you to the GEO page of the data, and clicking on the PMID will directly take you to the Pubmed page of the article. The default sorting order is "Sort by Default order", which can be copied to Excel for sorting, filtering, and other operations; the links will also be retained in the Excel file, and clicking on the link will directly take you to the paper webpage.

## 特殊情况的说明

1. 如果遇到TimeoutError，换个网络好的地方再试。

2. 偶尔会遇到影响因子那里都是NA的情况，可能是<https://www.scijournal.org>网站访问不畅，稍后重试即可。

3. 个别期刊在这个网站上检索不到影响因子，例如PNAS、NAR，所以会显示为NA。

4. 如果文章题目中出现特殊符号，例如“<”，题目会在此断掉，这是Entrez包的一个bug。

## Explanation of special circumstances
1. If you encounter a TimeoutError, try again in a place with a better network connection.
Occasionally, you may encounter a situation where all the impact factors are marked as NA. This may be due to poor access to the website <https://www.scijournal.org>. You can simply try again later.
3. Some journals, such as PNAS and NAR, cannot be searched for their impact factors on this website, so they will be displayed as NA.
4. If special symbols, such as "<", appear in the article title, the title will be broken here, which is a bug in the Entrez package.

In [ ]:
import IPython
print(IPython.sys_info())

!jupyter nbconvert --to html FigureYa104GEOmining.ipynb